### visualization script in Team3

In [ ]:
import open3d as o3d
import numpy as np

points_file = "/home/ailab/AILabDataset/01_Open_Dataset/39_AutoDna/3d_object_detection/3d_mod_av_db/points/0000000.npy"


# 1. 포인트 클라우드 로드
points = np.load(points_file)  # .npy 파일로부터 포인트 로드
print(points.shape)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points[:, :3])  # x, y, z 좌표만 가져옴

# RANSAC을 사용하여 바닥면 탐지
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01, ransac_n=3, num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

In [ ]:
points_file = "/home/ailab/AILabDataset/01_Open_Dataset/39_AutoDna/3d_object_detection/3d_mod_av_db/points/00100000.npy"



In [ ]:
import pickle 
kitti_path = "/home/ailab/AILabDataset/01_Open_Dataset/03_KITTI/kitti_dbinfos_train.pkl"

# 파일을 열고 데이터를 로드합니다
with open(kitti_path, 'rb') as file:
    data = pickle.load(file)

# 로드된 데이터를 확인합니다
print(data)


In [ ]:
import pickle 
custom_av_pkl = "/home/ailab/AILabDataset/01_Open_Dataset/39_AutoDna/3d_object_detection/3d_mod_av_db/Team_3_whole_dbinfos_train_origin2.pkl"

# 파일을 열고 데이터를 로드합니다
with open(custom_av_pkl, 'rb') as file:
    data_dna = pickle.load(file)

# 로드된 데이터를 확인합니다
print(data_dna)


In [ ]:
import os

def get_top_5_entries(folder_path):
    entries = os.listdir(folder_path)          # 폴더 내 항목 가져오기
    top_5_entries = []

    for i, entry in enumerate(entries):
        if i >= 5:                             # 5번까지만 반복
            break
        top_5_entries.append(entry)

    return top_5_entries

folder_path = '/home/ailab/AILabDataset/01_Open_Dataset/39_AutoDna/3d_object_detection/3d_mod_av_db/labels'
top_5_entries = get_top_5_entries(folder_path)
print("Top 5 entries:", top_5_entries)

### 모델 결과 시각화


In [ ]:
import argparse
import glob
from pathlib import Path

try:
    import open3d
    from visual_utils import open3d_vis_utils as V
    OPEN3D_FLAG = True
except:
    import mayavi.mlab as mlab
    from visual_utils import visualize_utils as V
    OPEN3D_FLAG = False

import numpy as np
import torch

from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.datasets import DatasetTemplate
from pcdet.models import build_network, load_data_to_gpu
from pcdet.utils import common_utils

def load_gt_data(label_path, index):
    """GT 데이터 로드 함수 - index.txt 파일 형식 (클래스 이름 포함)"""
    gt_file = Path(f"{label_path}/{index:08d}.txt")

    gt_data = []
    if gt_file.is_file():
        with open(gt_file, 'r') as f:
            for line in f:
                parts = line.strip().split()
                coords = list(map(float, parts[:-1])) # 나머지 요소는 좌표와 크기 정보 (float으로 변환)
                gt_data.append(coords) # 튜플로 저장 (클래스 이름, 좌표 정보)
        print(gt_data)
        return gt_data
    else:
        print("gt file does not exist")
    return None

label_path = "/home/ailab/AILabDataset/01_Open_Dataset/39_AutoDna/3d_object_detection/3d_mod_av_db/labels"

gt = load_gt_data(label_path, 1000)

### 벨리데이션 데이터셋 연결

In [ ]:
import numpy as np
import os

# 웨이모 데이터셋
npy_path = "/home/ailab/AILabDataset/01_Open_Dataset/30_Waymo/waymo_open_dataset_v_1_2_0/waymo_processed_data_v0_5_0/segment-366934253670232570_2229_530_2249_530_with_camera_labels/0000.npy"

data = np.load(npy_path)
data

In [ ]:
### KITTI데이터셋

import numpy as np
# .bin 파일 경로
file_path = '/home/ailab/AILabDataset/01_Open_Dataset/03_KITTI/train/lidar/000002.bin'

# .bin 파일을 numpy 배열로 로드 (포인트 하나당 4개의 float 값이 있다고 가정)
point_cloud = np.fromfile(file_path, dtype=np.float32).reshape(-1, 4)

# 결과 확인
print("Loaded Point Cloud Data:")
print(point_cloud)

In [ ]:
import argparse
import glob
from pathlib import Path
import pickle
try:
    import open3d
    from visual_utils import open3d_vis_utils as V
    OPEN3D_FLAG = True
except:
    import mayavi.mlab as mlab
    from visual_utils import visualize_utils as V
    OPEN3D_FLAG = False

import numpy as np
import torch

from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.datasets import DatasetTemplate
from pcdet.models import build_network, load_data_to_gpu
from pcdet.utils import common_utils


def load_gt_data(label_path, index):
    """GT 데이터 로드 함수 - index.txt 파일 형식 (클래스 이름 포함)"""
    gt_file = Path(f"{label_path}/{index:06d}.txt")

    gt_data = []
    if gt_file.is_file():
        with open(gt_file, 'r') as f:
            for line in f:

                parts = line.strip().split()
                coords = list(map(float, parts[4:-1])) # 나머지 요소는 좌표와 크기 정보 (float으로 변환)
                xmin, ymin, xmax, ymax = coords[4:7]
                location = coords[11:13]
                rotation = coords[14]
                coords = np.concat( (np.array(location),np.array([xmax-xmin, ymax-ymin, coords[8]]), np.array([rotation])) , axis = 0)
                gt_data.append(coords) # 튜플로 저장 (클래스 이름, 좌표 정보)
        print(gt_data)
        return np.array(gt_data)
    else:
        print("gt file does not exist")
    return None
    
    
pcl_path='/home/ailab/AILabDataset/01_Open_Dataset/03_KITTI/train/lidar'
index = 0
data = np.fromfile(pcl_path + f'/{index:06d}.bin', dtype=np.float32).reshape(-1,4)

data

In [ ]:
import pickle

path = "/home/ailab/AILabDataset/01_Open_Dataset/39_AutoDna/3d_object_detection/3d_mod_av_db/Team_3_whole_dbinfos_train_origin2.pkl"


with open(path, 'rb') as f:
    data = pickle.load(f)
data

In [ ]:
# .bin 파일 개수 세기
import os
directory_path = "/home/ailab/AILabDataset/01_Open_Dataset/39_AutoDna/3d_object_detection/3d_mod_av_db/Team_3_whole_gt_database_train_origin2"
count=0
for file in os.listdir(directory_path):
    if file.endswith('.bin'):   
        count +=1
file_count = len(count)

print(f"team3 총 {count}개의 .bin 파일이 있습니다.")

# .bin 파일 개수 세기
directory_path= "/home/ailab/AILabDataset/01_Open_Dataset/39_AutoDna/3d_object_detection/3d_mod_av_db/Team_1_gt_database"
count=0
for file in os.listdir(directory_path):
    if file.endswith('.bin'):   
        count +=1
file_count = len(count)

print(f"team1 총 {count}개의 .bin 파일이 있습니다.")
